In [ ]:
# !pip install sklearn, pennylane, matplotlib

# Comparing kernel methods and variational quantum circuits on a real dataset

In [2]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor
from pennylane.optimize import NesterovMomentumOptimizer

import matplotlib.pyplot as plt

/Users/gbltostibalduc/miniforge3/envs/plTutorialsEnv/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Declare a random number generator for reproducibility

In [3]:
rng = np.random.default_rng(42)

## Loading and preparing the Iris dataset

In [4]:
X, y = load_iris(return_X_y=True)

# pick only the first two classes (corresponding to the first 100 samples)
X = X[:100]
y = y[:100]

# scaling the data for better training and avoiding problems due to periodic encoding
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# move labels from [0, 1] to [-1, 1] range
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)


[[ 8.28527933e-01  6.31902691e-01  1.13639313e+00  1.44757384e+00]
 [-7.37687441e-01 -2.07835104e-01 -8.74308565e-01 -1.04211089e+00]
 [-8.94308978e-01 -1.46744180e+00  3.04378636e-01  3.80566095e-01]
 [ 4.54202458e-02 -1.67737625e+00  7.89720424e-01  9.14069966e-01]
 [ 4.54202458e-02  8.41837140e-01 -1.08231219e+00 -1.04211089e+00]
 [ 1.92487869e+00  2.09934449e-03  1.27506221e+00  1.26973921e+00]
 [-5.81065904e-01  1.47164049e+00 -6.66304941e-01 -6.86441647e-01]
 [-4.24444366e-01  8.41837140e-01 -9.43643106e-01 -1.04211089e+00]
 [ 9.85149470e-01 -2.07835104e-01  1.20572767e+00  1.09190459e+00]
 [-1.36417359e+00  2.12033793e-01 -1.01297765e+00 -1.04211089e+00]
 [ 9.85149470e-01 -4.17769553e-01  1.27506221e+00  1.09190459e+00]
 [-7.37687441e-01  2.12033793e-01 -1.15164673e+00 -1.04211089e+00]
 [ 4.54202458e-02 -1.25750735e+00  7.89720424e-01  9.14069966e-01]
 [-1.52079513e+00 -1.67737625e+00 -1.08231219e+00 -8.64276271e-01]
 [ 8.28527933e-01 -1.88731069e+00  7.89720424e-01  3.80566095e

## Angle encoding

Exact amplitude encoding may require up to an exponential amount of operations and thus it does not work for NISQ. We can do another type of encoding, i.e. *angle encoding*, for which the features are encoded in as many qubits and each of them is used as a rotation angle of a gate.

In [5]:
n_qubits = len(X_train[0])

Let's create a function that encodes each of the features as a $R_Z$ rotation for each qubit

In [6]:
def angle_encoding_rz(x):
    """Encodes features as Z-rotations

    Args:
        x (np.ndarray): data point where the entries are the features to be encoded
    """
    # ================
    # YOUR CODE BELOW
    # ================

## Computing the kernel

In quantum kernel methods, the quantum computer is used exclusively to compute the kernel matrix. 

Remember that the kernel is given by the inner product of two samples $x_1$ and $x_2$. More correctly, it is the inner product between their mappings in the higher-dimensional feature space, thus $\phi(x_1)$ and $\phi(x_2)$. For us, this higher-dimensional mapping is given by our $R_Z$-encoding, which leads to the states $|\phi(x_1)\rangle$ and $|\phi(x_2)\rangle$. Therefore, we want to compute $|\langle \phi(x_2) | \phi(x_1) \rangle|^2$ (the square matters because the kernel is a measure of a distance).

It is easy to verify that the circuit used to compute the kernel is

<img src="kernel.png" alt="kernel" width="400"/>

where $\Phi(x)$ computes the feature map and the final measuerement is on the all-zeros projector $|0\dots 0\rangle\langle 0\dots 0|$ (equivalent to computing the probability of measuring the all-zeros state).

In [7]:
dev_1 = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2 ** n_qubits, 2 ** n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_1)
def kernel(x1, x2):
    """Computes the kernel with the R_Z feature map

    Args:
        x1 (np.ndarray): first sample
        x2 (np.ndarray): second sample

    Returns:
        float: kernel value
    """
    # ================
    # YOUR CODE BELOW
    # ================
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [7]:
kernel(X_train[0], X_train[0])

tensor(1., requires_grad=True)

## Training the support vector machine (SVM)

Once we have a way to compute the kernel, we can use it to train a support vector machine, which is a classifier. This requires to solve an optimization problem to determine the parameters of the so-called *dual-problem*, which in tuern allow to make predictions for new samples.

This procedure is easily taken care of by the scikit-learn library, provided that we build a function for the kernel **matrix**. This has a syntax where the two set of samples can belong to two different datasets A and B, even though, in our case A=B.

In [9]:
def kernel_matrix(A, B):
   """Returns the kernel matrix from entries in two datasets

   Args:
       A (List[np.ndarray]): first dataset
       B (List[np.ndarray]): second dataset

   Returns:
       np.ndarray: kernel matrix
   """
   return np.array([[kernel(a, b) for b in B] for a in A])

In [10]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

### Accuracy of the results

The trained SVM can now be used on unseen samples (test data-set) to see how it performs in predicting their labels.

In [11]:
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

1.0

## Training with a variational classifier

As for the parity training, here we will be using a variational quantum classifier to classify the Iris dataset. After training, we will check the accuracy score and compare it with the kernel method

In [8]:
@qml.qnode(dev_1)
def vqc_circuit(theta, x):
    """Returns the variational quantum circuit with angle embedding and strongly entangling layers
    as ansatz

    Args:
        theta (tensor_like): trainable ansatz parameters. The first dimension determines the number of
            layers. Shape = (L, M ,3); L = layers, M = n_qubits, 3 = rotations per qubit.
        x (np.ndarray): sample (properly scaled)

    Returns:
        float: prediction of the quantum model
    """

    # embedding
    AngleEmbedding(x, wires=range(n_qubits))

    # trainable measurement
    StronglyEntanglingLayers(theta, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

In [9]:
def vqc_with_bias(theta, bias, x):
    """Adds bias to the outcome of a vqc

    Args:
        theta (tensor_like): trainable ansatz parameters
        bias (float): classical bias
        x (np.ndarray): sample (properly scaled)

    Returns:
        float: (vqc output) + bias
    """
    return vqc_circuit(theta, x) + bias

### Cost function

In [10]:
def square_loss(labels, predictions):
    """Computes the MSE between labels and predictions

    Args:
        labels (List[int]): actual values
        predictions (List[int]): model predictions

    Returns:
        float: value of the MSE
    """
    loss = 0.
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2

    loss /= len(labels)
    return loss

In [11]:
def cost(theta, bias, X, y):
    """Computes the predictions and returns the MSE over the dataset

    Args:
        theta (List[np.ndarray]): all the variational parameters, one
            array per layer
        bias (float): classical bias value, added to the output of the quantum circuit
        X (List[List]): list of all train samples
        y (List): labels

    Returns:
        float: MSE over the dataset
    """
    predictions = [vqc_with_bias(theta, bias, x) for x in X]
    return square_loss(y, predictions)

### Hyperparameters and initial values

In [14]:
n_layers = 6
n_iters = 100

theta_init = .01 * rng.normal(size=(n_layers, n_qubits, 3), requires_grad=True)
print(theta_init)
bias_init = np.array(0., requires_grad=True)
# print(bias_init)

[[[ 0.0015854   0.0062559  -0.00309347]
  [ 0.00456775 -0.00661926 -0.00363054]
  [-0.00381738 -0.0119584   0.00486972]
  [-0.00469402  0.00012494  0.00480747]]

 [[ 0.00446531  0.00665385 -0.00098485]
  [-0.00423298 -0.00079718 -0.01687334]
  [-0.01447112 -0.013227   -0.00997247]
  [ 0.00399774 -0.00905479 -0.00378163]]

 [[ 0.01299228 -0.00356264  0.00737516]
  [-0.00933618 -0.00205438 -0.00950022]
  [-0.00339033  0.00840308 -0.0172732 ]
  [ 0.00434424  0.00237736 -0.0059415 ]]

 [[-0.01446058  0.0007213  -0.00529493]
  [ 0.00232676  0.00021852  0.01601779]
  [-0.00239356 -0.01023497  0.00179276]
  [ 0.00219997  0.01359188  0.00835111]]

 [[ 0.00356871  0.01463303 -0.01188763]
  [-0.00639752 -0.00926576 -0.0038981 ]
  [-0.01376686  0.00635151 -0.00222223]
  [-0.01470806 -0.01015579  0.00313514]]

 [[ 0.00838127  0.01996731  0.02913862]
  [ 0.00414409 -0.00989538 -0.02132046]
  [ 0.00267711 -0.00812941 -0.00415357]
  [-0.00612097 -0.00140791  0.0106598 ]]]


### Optimize

In [19]:
opt = NesterovMomentumOptimizer(0.01)
batch_size = 10

# train the variational classifier
theta = theta_init
bias = bias_init
for it in range(n_iters):

    # Update the weights for each optimizer step
    batch_index = rng.integers(0, len(X_train), size=batch_size)
    X_batch = np.array(X_train[batch_index])
    y_batch = np.array(y_train[batch_index])
    theta, bias, _, _ = opt.step(cost, theta, bias, X_batch, y_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(vqc_with_bias(theta, bias, x)) for x in X_train]
    predictions_test = [np.sign(vqc_with_bias(theta, bias, x)) for x in X_test]

    # Compute accuracy on train and test set
    acc_train = accuracy_score(y_train, predictions_train)
    acc_test = accuracy_score(y_test, predictions_test)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(theta, bias, X_train, y_train), acc_train, acc_test)
    )

Iter:     1 | Cost: 1.0769043 | Acc train: 0.4400000 | Acc validation: 0.3600000 
Iter:     2 | Cost: 1.0748452 | Acc train: 0.4533333 | Acc validation: 0.3600000 
Iter:     3 | Cost: 1.0722913 | Acc train: 0.4533333 | Acc validation: 0.3600000 
Iter:     4 | Cost: 1.0688859 | Acc train: 0.4666667 | Acc validation: 0.3600000 
Iter:     5 | Cost: 1.0630089 | Acc train: 0.4800000 | Acc validation: 0.3600000 
Iter:     6 | Cost: 1.0588640 | Acc train: 0.4666667 | Acc validation: 0.4000000 
Iter:     7 | Cost: 1.0554286 | Acc train: 0.4800000 | Acc validation: 0.4000000 
Iter:     8 | Cost: 1.0547370 | Acc train: 0.4800000 | Acc validation: 0.4400000 
Iter:     9 | Cost: 1.0517812 | Acc train: 0.4800000 | Acc validation: 0.4400000 
Iter:    10 | Cost: 1.0496193 | Acc train: 0.4800000 | Acc validation: 0.4400000 
Iter:    11 | Cost: 1.0513859 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:    12 | Cost: 1.0500976 | Acc train: 0.4933333 | Acc validation: 0.6000000 
Iter:    13 | Co